In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
...

driver = webdriver.Chrome('C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UC5jHGolFUNHe0HsCJEKbb3g/videos')

#여기서부터
num_of_pagedowns=100
body = driver.find_element_by_xpath('/html/body')

#자동으로 스크롤을 내려서 더이상 영상이 로드 안될때까지 내려줌.
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)
    num_of_pagedowns -= 1
    try:
        driver.find_element_by_xpath("""//*[@id="feed-main-what_to_watch"]/button""").click()
    except:
        None
        
page = driver.page_source
page

In [ ]:
from bs4 import BeautifulSoup
#태그, 클래스
soup = BeautifulSoup(page,'lxml')

all_title = soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')
title = [soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')[n].string for n in range(0,len(all_title))]
title

In [ ]:
from bs4 import BeautifulSoup

...


soup = BeautifulSoup(page,'lxml')

#채널명('태그','class명')
chennel = soup.find('yt-formatted-string','style-scope ytd-channel-name').string
#구독자 수
sub_num = soup.find('yt-formatted-string','style-scope ytd-c4-tabbed-header-renderer').string
#조회수, 올린지 얼마나 되었는지(업로드 시점)
c = soup.find_all('span','style-scope ytd-grid-video-renderer')
view_num = [soup.find_all('span','style-scope ytd-grid-video-renderer')[n].string for n in range(0,len(c))]
view_num

In [ ]:
from time import localtime, strftime

...

#현재 시간
extract_date = strftime("%Y/%m/%d %H:%M:%S", localtime())
extract_date

In [ ]:
...

youtube_video_list = []

x = 0		#조회수의 index
y = 1		#업로드 시점의 index

for i in range(0,len(all_title)):
    roww = []
    roww.append(title[i])
#     roww.append(video_time[i].strip())
    roww.append(chennel)
    roww.append(sub_num)
    roww.append(view_num[x])
    x += 2		#조회수만 append
    roww.append(view_num[y])
    y += 2		#업로드 시점만 append
    roww.append(extract_date)
    youtube_video_list.append(roww)			#2차원 list로 만들어줌
    
youtube_video_list

In [ ]:
import csv
#-*- coding: utf-8 -*-
...

csvfile = open("C:\\Users\\jmlee\\Code\\Hyoning.txt","w",-1,"utf-8")
csvwriter = csv.writer(csvfile)
for row in youtube_video_list:
    csvwriter.writerow(row)
csvfile.close()

# 후처리

In [1]:
#python3 이용
import os 
import sys
os.environ["SPARK_HOME"]=os.path.join(os.path.expanduser("~"),"Downloads\spark-2.4.2-bin-hadoop2.7")

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.7-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [3]:
HyoningRdd=spark.sparkContext.textFile(os.path.join("crawling txt","Hyoning.txt"))

In [4]:
HyoningList=HyoningRdd.collect()
print(type(HyoningList))

<class 'list'>


In [5]:
#/제거
import re
a=list()

for i in HyoningList:
    text=re.sub('/',' ',i)
    a.append(text)
    #print(text)
print(a)  


['감자 당면 듬뿍 넣은 매콤한 닭볶음탕과 총각김치 먹방 Spicy Braised Chicken Kimchi Korean food mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 3.7천회,1시간 전,2019 11 17 22:29:54', '', '따끈한 잔치국수와 꼬마김밥 먹방 Koreanfood Janchi Guksu & Gimbap Kimchi mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 4.7만회,1일 전,2019 11 17 22:29:54', '', '대패삼겹살로 만든 콩나물 불고기 계란찜 먹방 Spicy Bean sprout& Pork belly bulgogi mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 6.4만회,5일 전,2019 11 17 22:29:54', '', '토핑가득 페퍼로니피자 핫불고기피자 먹방 Hot Bulgogi & Pepperoni  Pizza mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 6.7만회,1주 전,2019 11 17 22:29:54', '', '얼큰한 닭칼국수 모듬군만두 배추김치 먹방 Spicy Chicken soup noodles Dumpling Kimchi mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 11만회,1주 전,2019 11 17 22:29:54', '', '매콤한 낙지덮밥과 김치콩나물국 먹방 Spicy Octopus with rice Kimchi Bean sprout soup mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 5.9만회,1주 전,2019 11 17 22:29:54', '', '부대찌개 라면사리 용가리치킨 오이도라지초무침 먹방 Budae jjigae koreanfood (spicy sausage stew) mukbang Eating sound,효닝 Hyoning,구독자 15만명,조회수 14만

In [6]:
#한글만 남기기
b=[]
for j in a:
    text=re.compile('[^ ㄱ-ㅣ가-힣]+') 
    result = text.sub(' ', j)
    b.append(result)
print(b)
print(type(b))
#     text=re.sub('[^\u3131-\u3163\uac00-\ud7ad]+',' ',i)

['감자 당면 듬뿍 넣은 매콤한 닭볶음탕과 총각김치 먹방                  효닝  구독자  만명 조회수  천회 시간 전       ', '', '따끈한 잔치국수와 꼬마김밥 먹방                  효닝  구독자  만명 조회수  만회 일 전       ', '', '대패삼겹살로 만든 콩나물 불고기 계란찜 먹방                  효닝  구독자  만명 조회수  만회 일 전       ', '', '토핑가득 페퍼로니피자 핫불고기피자 먹방                 효닝  구독자  만명 조회수  만회 주 전       ', '', '얼큰한 닭칼국수 모듬군만두 배추김치 먹방                  효닝  구독자  만명 조회수  만회 주 전       ', '', '매콤한 낙지덮밥과 김치콩나물국 먹방                      효닝  구독자  만명 조회수  만회 주 전       ', '', '부대찌개 라면사리 용가리치킨 오이도라지초무침 먹방                  효닝  구독자  만명 조회수  만회 주 전       ', '', '통베이컨을 올린 불닭크림파스타 통피클 먹방                  효닝  구독자  만명 조회수  만회 주 전       ', '', '양념순살게장 비빔밥과 비지찌개 배추김치 먹방                  효닝  구독자  만명 조회수  만회 주 전       ', '', '  신메뉴 새콤달콤한 블랙올리브치킨 뿌링소떡 빨간소떡 치즈볼 먹방              효닝  구독자  만명 조회수  만회 주 전       ', '', '스팸김치찌개와 오이무침 비엔나소세지 먹방              효닝  구독자  만명 조회수  만회 주 전       ', '', '얼굴만한 천원짜리 튀김과 치즈 고구마무스 듬뿍 신전떡볶이 고추튀김 김말이 먹방             효닝  구독자  만명 조회수  만회 주 전       ', '', '등갈비찜에 당면사리 총각김치 먹방          

In [7]:
HyoningRdd_=spark.sparkContext.parallelize(b)  

In [8]:
#불용어제거하고 빈도수 추출
stopwords = [u'리얼사운드',u'먹방',u'만회',u'구독자',u'만명',u'전',u'이공삼',u'천회',u'조회수',u'개월',u'년',u'대왕',u'주',u'같이',u'종',u'효닝',u'넣은',u'듬뿍',u'밥도둑',u'가득',u'천회년',u'만회년',u'신메뉴',u'너무',u'통통한',u'올린',u'맛있는',u'함께',u'맛집']
Hyoningstop=HyoningRdd_.flatMap(lambda x:x.split()).filter(lambda x:x not in stopwords)
print(type(Hyoningstop))

<class 'pyspark.rdd.PipelinedRDD'>


In [9]:
Hyoningstop=Hyoningstop.collect()
print(Hyoningstop)
print(type(Hyoningstop))

['감자', '당면', '매콤한', '닭볶음탕과', '총각김치', '시간', '따끈한', '잔치국수와', '꼬마김밥', '일', '대패삼겹살로', '만든', '콩나물', '불고기', '계란찜', '일', '토핑가득', '페퍼로니피자', '핫불고기피자', '얼큰한', '닭칼국수', '모듬군만두', '배추김치', '매콤한', '낙지덮밥과', '김치콩나물국', '부대찌개', '라면사리', '용가리치킨', '오이도라지초무침', '통베이컨을', '불닭크림파스타', '통피클', '양념순살게장', '비빔밥과', '비지찌개', '배추김치', '새콤달콤한', '블랙올리브치킨', '뿌링소떡', '빨간소떡', '치즈볼', '스팸김치찌개와', '오이무침', '비엔나소세지', '얼굴만한', '천원짜리', '튀김과', '치즈', '고구마무스', '신전떡볶이', '고추튀김', '김말이', '등갈비찜에', '당면사리', '총각김치', '네네', '핫블링', '양념치킨', '계란감자', '샐러드빵', '보쌈과', '아삭아삭', '보쌈김치', '매콤한', '쟁반국수', '스팸', '깍두기', '김치', '볶음밥', '신제품', '불타는', '고추짜장', '맵다', '고향만두', '총각김치', '진한', '카레', '칼국수', '코코넛', '새우튀김', '총각김치', '페퍼로니피자', '핫쉬림프피자', '갈릭딥핑소스', '피자샵', '매콤한', '국물', '닭발', '떠먹기', '매콤똥집튀김', '주먹밥', '차돌박이', '고추장찌개', '돈까스', '집밥', '총각김치', '매콤한', '빨간찜닭', '납작당면', '추가', '파김치', '시원', '칼칼한', '짬뽕밥과', '탕수육', '파김치', '김치우동과', '토핑가득', '유부초밥', '파삭파삭한', '꽈배기', '치즈핫도그', '팥도넛', '옥수수', '고구마', '뜨끈한', '소고기', '미역국에', '밥', '말아서', '배추김치', '계란말이', '비오는', '날엔', '다', '있는', '모듬전과', '파절이', '총각김치',

In [10]:
HyoningstopRdd=spark.sparkContext.parallelize(Hyoningstop) 

In [11]:
#정규화한다음 빈도수 추출
Hyoning=HyoningstopRdd\
    .flatMap(lambda x:x.split(','))\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(30)

In [12]:
print(Hyoning)

[(26, '매콤한'), (22, '김치'), (18, '총각김치'), (18, '바삭한'), (17, '얼큰한'), (16, '파김치'), (15, '집밥'), (14, '배추김치'), (14, '치킨'), (12, '한식'), (11, '시원한'), (9, '계란말이'), (8, '깍두기'), (8, '새우튀김'), (8, '바삭바삭'), (7, '스팸'), (7, '주먹밥'), (7, '계란후라이'), (7, '중국당면'), (6, '치즈볼'), (6, '치즈'), (6, '보쌈김치'), (6, '쟁반국수'), (6, '탕수육'), (6, '떡갈비'), (6, '떡볶이'), (6, '짜장면'), (6, '달콤한'), (6, '라면'), (6, '매운')]


In [13]:
#파일저장
f=open("C:\\Users\\jmlee\\Code\\Frequency.txt",'a',-1,"utf-8")
A=len(Hyoning)
print (A)
for i in range (0,A):
    f.writelines(str(Hyoning[i]))
    f.writelines("\n")
f.close()

30
